In [55]:
import pandas as pd
from wgs_qc_utils.reader import read_variant_calls
import yaml
from single_cell.utils import inpututils
import os
import pysam

dlp_museq_file = '/work/shah/tantalus/SC-3804/results/variant_calling/sample_SA1255LA/museq.vcf.gz'
dlp_strelka_file = '/work/shah/tantalus/SC-3804/results/variant_calling/sample_SA1255LA/strelka_snv.vcf.gz'

dlp_cell_bams_dir = "/juno/work/shah/isabl_data_lake/analyses/24/96/2496/bams"
dlp_cell_bams = [os.path.join(dlp_cell_bams_dir, f.strip()) 
                 for f in inpututils.load_yaml('/juno/work/shah/isabl_data_lake/analyses/24/96/2496/bams/metadata.yaml')["filenames"] if not f.endswith("bai")]

wgs_museq_file = '/juno/work/shah/isabl_data_lake/analyses/24/72/2472/results/copynumber/SPECTRUM-OV-022_S1_LEFT_OVARY/titan/SPECTRUM-OV-022_S1_LEFT_OVARY_museq.vcf'
wgs_strelka_file = '/juno/work/shah/isabl_data_lake/analyses/24/65/2465/results/variants/SPECTRUM-OV-022_S1_LEFT_OVARY/SPECTRUM-OV-022_S1_LEFT_OVARY_strelka_snv_annotated.vcf.gz'

wgs_normal_bam = '/juno/work/shah/isabl_data_lake/analyses/24/22/2422/results/SPECTRUM-OV-022_BC1_NORMAL_NORMAL/SPECTRUM-OV-022_BC1_NORMAL_NORMAL.bam'
wgs_tumour_bam = '/juno/work/shah/isabl_data_lake/analyses/24/48/2448/results/SPECTRUM-OV-022_S1_LEFT_OVARY_TUMOR/SPECTRUM-OV-022_S1_LEFT_OVARY_TUMOR.bam'





In [56]:
def get_vaf(bam_path, chrom, pos, alt, ref):
    bam = pysam.AlignmentFile(bam_path)
    counts = {'A':0, 'C':0, 'G':0, 'T':0, 'N':0}
    for pileupcolumn in bam.pileup(chrom, pos-200, pos+200):
        for pileupread in pileupcolumn.pileups:
            if pileupcolumn.pos == pos-1:
                base = pileupread.alignment.query_sequence[pileupread.query_position]
                counts[base]+=1
    return counts[ref], counts[alt]

In [67]:
dlp_museq = read_variant_calls.read_with_tumour(dlp_museq_file)
dlp_strelka = read_variant_calls.read_with_tumour(dlp_strelka_file)

# wgs_museq = read_variant_calls.read_with_tumour(wgs_museq_file)
# wgs_strelka = read_variant_calls.read_with_tumour(wgs_strelka_file)


In [ ]:
wgs_museq["caller"] = ["mutationseq"] * len(wgs_museq.index)
wgs_museq["score"] = wgs_museq["info"].str.split(";")
wgs_museq["score"] = wgs_museq.score.apply(lambda score: score[0])

wgs_strelka["caller"] = ["strelka"] * len(wgs_strelka.index)
wgs_strelka["score"] = wgs_strelka["info"].str.split(";")
wgs_strelka["score"] = wgs_strelka.score.apply(lambda score: score[0])

WGS = wgs_museq.merge(wgs_strelka, how='outer', on=['chr', 'pos', 'id', 'ref', 'alt', 'qual','filter', 'info', 'caller', 'score'])[['chr', 'pos', 'id', 'ref', 'alt', 'qual','filter', 'caller', 'score']]

WGS["datatype"] = ["WGS"] * len(WGS.index)

In [68]:
dlp_museq["museq_caller"] = [True] * len(dlp_museq.index)
dlp_museq["museq_score"] = dlp_museq["info"].str.split(";")
dlp_museq["museq_score"] = dlp_museq.museq_score.apply(lambda museq_score: museq_score[0])
dlp_museq = dlp_museq[['chr', 'pos', 'id', 'ref', 'alt', 'qual','filter', 'museq_caller', 'museq_score']]

dlp_strelka["strelka_caller"] = [True] * len(dlp_strelka.index)
dlp_strelka["strelka_score"] = dlp_strelka["info"].str.split(";")
dlp_strelka["strelka_score"] = dlp_strelka.strelka_score.apply(lambda strelka_score: strelka_score[0])
dlp_strelka = dlp_strelka[['chr', 'pos', 'id', 'ref', 'alt', 'qual','filter', 'strelka_caller', 'strelka_score']]

DLP = dlp_museq.merge(dlp_strelka, how='outer', on=['chr', 'pos', 'ref', 'alt', 'id', 'qual', 'filter'])
DLP["datatype"] = ["DLP"] * len(DLP.index)
DLP[]


,chr,pos,id,ref,alt,qual,filter,museq_caller,museq_score,strelka_caller,strelka_score,datatype
0,1,10492,.,C,T,3.1,INDL,True,PR=0.51,NaN,NaN,DLP
1,1,76106,.,T,G,4.08,PASS,True,PR=0.61,NaN,NaN,DLP
2,1,531854,.,T,G,4.1,INDL,True,PR=0.61,NaN,NaN,DLP
3,1,536195,.,T,C,4.14,PASS,True,PR=0.61,NaN,NaN,DLP
4,1,546732,.,C,T,4.63,PASS,True,PR=0.66,NaN,NaN,DLP
...,...,...,...,...,...,...,...,...,...,...,...,...
137562,X,155059985,.,A,G,.,.,NaN,NaN,True,QSS=41,DLP
137563,X,155198866,.,T,A,.,.,NaN,NaN,True,QSS=26,DLP
137564,X,155246353,.,C,T,.,.,NaN,NaN,True,QSS=73,DLP
137565,X,155252882,.,A,G,.,.,NaN,NaN,True,QSS=16,DLP


In [30]:
UNION = WGS.merge(DLP, how='outer', on = ['chr', 'pos', 'id', 'ref', 'alt', 'qual', 'filter', 'caller', 'score', 'datatype'])

In [34]:
UNION

,chr,pos,id,ref,alt,qual,filter,caller,score,datatype
0,1,10622,.,T,G,8.32,INDL,mutationseq,PR=0.85,WGS
1,1,14653,.,C,T,10.14,PASS,mutationseq,PR=0.90,WGS
2,1,14907,.,A,G,10.32,PASS,mutationseq,PR=0.91,WGS
3,1,14930,.,A,G,11.05,PASS,mutationseq,PR=0.92,WGS
4,1,16068,.,T,C,9.41,PASS,mutationseq,PR=0.89,WGS
...,...,...,...,...,...,...,...,...,...,...
2403956,X,155059985,.,A,G,.,.,strelka,QSS=41,DLP
2403957,X,155198866,.,T,A,.,.,strelka,QSS=26,DLP
2403958,X,155246353,.,C,T,.,.,strelka,QSS=73,DLP
2403959,X,155252882,.,A,G,.,.,strelka,QSS=16,DLP


In [27]:
UNION.to_csv("/juno/work/shah/abramsd/CODE/wgs_qc_utils/union_set_snvs_022_WGS_DLP.csv", sep="\t", index=False, header=True)

In [23]:
UNION

,chr,pos,id,ref,alt,qual,filter,caller,score
0,1,10622,.,T,G,8.32,INDL,mutationseq,PR=0.85
1,1,14653,.,C,T,10.14,PASS,mutationseq,PR=0.90
2,1,14907,.,A,G,10.32,PASS,mutationseq,PR=0.91
3,1,14930,.,A,G,11.05,PASS,mutationseq,PR=0.92
4,1,16068,.,T,C,9.41,PASS,mutationseq,PR=0.89
...,...,...,...,...,...,...,...,...,...
2403697,X,155059985,.,A,G,.,.,strelka,QSS=41
2403698,X,155198866,.,T,A,.,.,strelka,QSS=26
2403699,X,155246353,.,C,T,.,.,strelka,QSS=73
2403700,X,155252882,.,A,G,.,.,strelka,QSS=16


In [26]:
testbam = dlp_cell_bams[0]
get_vaf(testbam, UNION["chr"][0],  UNION["pos"][0],  UNION["alt"][0],  UNION["ref"][0])

(0, 0)

1 10622 G T
